## Reed-Muller Code:

In [1]:
class ReedMullerCode:
    
    def __init__(self, r, m, q):
        
        if not (q == 2):
            raise ValueError('Works only in binary case')
            
        self.r = r
        self.m = m
        self.q = q
        
        self.n = 2**self.m
        self.k = sum(binomial(self.m,i) for i in range(self.r + 1))
        self.d = 2**(self.m - self.r)
        
        # Initializing field
        self.FF = FiniteField(q)
        
        self.G = matrix(self.FF, self.k, self.n, lambda i,j : 0)
        
        print('n, k, d:')
        print(self.n,self.k,self.d)
        self.G = self.get_G(self.r, self.m)
        print(self.G)
            
    def get_G(self, r, m):
        n = 2**m
        k = sum(binomial(m,i) for i in range(r + 1))
        d = 2**(m - r)

        if (n == k):
            print('trivial code')
            return matrix(self.FF, k, n, lambda i,j : 1) #???

        elif (n == d):
            print('repetition code')
            return matrix(self.FF, k, n, lambda i,j : 1)

        elif (d == 2 and n == k + 1):
            print('single-parity check code')
            PCC = codes.ParityCheckCode(GF(self.q), k)
            return PCC.generator_matrix()

        else:
            print('recursion is needed')
            return block_matrix([ [self.get_G(r,m-1), self.get_G(r,m-1)], [0, self.get_G(r-1,m-1)] ])
            

In [2]:
C = ReedMullerCode(2,4,2)

n, k, d:
16 11 4
recursion is needed
single-parity check code
single-parity check code
recursion is needed
single-parity check code
single-parity check code
repetition code
[1 0 0 0 0 0 0 1|1 0 0 0 0 0 0 1]
[0 1 0 0 0 0 0 1|0 1 0 0 0 0 0 1]
[0 0 1 0 0 0 0 1|0 0 1 0 0 0 0 1]
[0 0 0 1 0 0 0 1|0 0 0 1 0 0 0 1]
[0 0 0 0 1 0 0 1|0 0 0 0 1 0 0 1]
[0 0 0 0 0 1 0 1|0 0 0 0 0 1 0 1]
[0 0 0 0 0 0 1 1|0 0 0 0 0 0 1 1]
[---------------+---------------]
[0 0 0 0 0 0 0 0|1 0 0 1 1 0 0 1]
[0 0 0 0 0 0 0 0|0 1 0 1 0 1 0 1]
[0 0 0 0 0 0 0 0|0 0 1 1 0 0 1 1]
[0 0 0 0 0 0 0 0|0 0 0 0 1 1 1 1]
